In [1]:
import os
import sys
# sys.path.insert(0, os.path.relpath('../..'))
sys.path.insert(0, os.path.relpath('../../../../..')) 

In [2]:
from recidiviz.calculator.modeling.population_projection.super_simulation import SuperSimulation
from recidiviz.calculator.modeling.population_projection.incarceration_transitions import IncarceratedTransitions
from recidiviz.calculator.modeling.population_projection.release_transitions import ReleasedTransitions
from recidiviz.calculator.modeling.population_projection.spark_policy import SparkPolicy
import pandas as pd
from functools import partial
import yaml

In [5]:
test = open('../state/AZ/az_state_prison_2808_model_inputs.yaml')
test_sim = SuperSimulation(test)
test_sim.simulate_baseline('pre-trial', 'prison')

ValueError: Max sentence length is not set